In [1]:
"""
Imports Here
"""
from dataset import trainloader
from dataset import testloader
from dataset import trainshape
from dataset import testshape

/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: 'dlopen(/Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so, 6): Library not loaded: @rpath/libpng16.16.dylib
  Referenced from: /Users/oliver/opt/anaconda3/lib/python3.9/site-packages/torchvision/image.so
  Reason: Incompatible library version: image.so requires version 56.0.0 or later, but libpng16.16.dylib provides version 54.0.0'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
from model import VisionTransformer
from model import Attention
from model import TransBlock

In [3]:
TRAIN_LOSS = []
TRAIN_ACC = []

def train(model, dataloader, lossfunc, lr=0.1, momentum=0.9, batch_size=16, nepochs=10):
    device = next(net.parameters()).device # check what device the net parameters are on
    optimizer = optim.Adam(net.parameters(), weight_decay=1e-5, lr=lr)
    
    """training"""
    for i in range(nepochs): # for each epoch
        epoch_loss = 0
        model.train()
        n_batches = 0
        for (x_batch, y_batch) in dataloader: # for each mini-batch
            optimizer.zero_grad()
            loss = lossfunc(model.forward(x_batch), y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss
            n_batches += 1
        epoch_loss /= n_batches
    
        """evaluating"""
        model.eval()
        accuracy = test(net, x, y, batch_size=batch_size)

        """get performance"""
        TRAIN_LOSS.append(epoch_loss)
        TRAIN_ACC.append(accuracy)

def test(model, dataloader, batch_size=1):
    with torch.no_grad(): # disable automatic gradient computation for efficiency
        device = next(net.parameters()).device
        
        """make predictions"""
        pcls = []
        items = 0
        for x, y in dataloader:
            x = x.to(device)
            torch.max(model(x), 1)[1].cpu()
            tcls = torch.max(y, 1)[1].cpu()
            pcls.append(tcls)
            items += 1

        """get accuracy"""
        pcls = torch.cat(pcls) # concat predictions on the mini-batches
        accuracy = pcls.sum().float() / items
        return accuracy

In [4]:
shape = trainshape()

In [5]:
print(shape)

torch.Size([240, 256])


In [12]:
batchsize=16
Np, P = trainshape()
model = VisionTransformer(inputsize=(batchsize, Np, P), nblocks=2)

In [13]:
trainloader1 = trainloader(batchsize=batchsize)

In [16]:
import time
import torch
import torch.nn as nn
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimiser = optim.AdamW(model.parameters(), lr=1e-4)

tensor(1.5862, grad_fn=<NllLossBackward0>)
90.11283302307129


In [20]:
TIMES = []
LOSS = []
i = 0
criterion.zero_grad()
out = model(x)
loss = criterion(out, y.type(torch.LongTensor))
print(loss)
time0 = time.time()
for (x, y) in trainloader1:
    #time1 = time.time()
    criterion.zero_grad()
    out = model(x)
    loss = criterion(out, y.type(torch.LongTensor))
    LOSS.append(loss.item())
    loss.backward()
    optimiser.step()
    #time2 = time.time()
    #TIMES.append(time2-time1)
time1 = time.time()
print(time1-time0)
#print("TIMES:", TIMES)
#print("LOSS:", LOSS)

tensor(0.7551, grad_fn=<NllLossBackward0>)
97.1146309375763


In [21]:
criterion.zero_grad()
out = model(x)
loss = criterion(out, y.type(torch.LongTensor))
print(loss)

tensor(0.7144, grad_fn=<NllLossBackward0>)


In [23]:
print(sum(param.numel() for param in model.parameters()))

62530


In [24]:
for param in model.parameters():
    print(param.numel())

64
16384
64
64
64
64
64
12288
192
4096
64
4096
4096
4096
8192
128
8192
64
64
64
128
2
